In [18]:

# install libraries 
!pip install pandas
!pip install zipfile
!pip install kaggle
!pip install openpyxl

# import the pandas library
import pandas as pd

# import zipfile library (we will use this to extract the file downloaded from Kaggle)
import zipfile

# import kaggle library (we will use this to download the dataset programatically from Kaggle)
import kaggle


ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


     -------------------------------------- 250.0/250.0 kB 5.1 MB/s eta 0:00:00


In [6]:
# Downloading dataset from kaggle using kaggle API 
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset


  0%|          | 0.00/165k [00:00<?, ?B/s]
100%|██████████| 165k/165k [00:00<00:00, 3.90MB/s]


In [7]:
# extracting file from downloaded zip 
zipfile_name = 'london-bike-sharing-dataset.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file: 
    file.extractall()

In [8]:
# Utilizing Pandas DF to read the csv
bikes = pd.read_csv("london_merged.csv")

In [9]:
# Looking into bikes DF (Basic summary information/ row & column information ) 
bikes.info()
bikes.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


(17414, 10)

In [10]:
#Counting the Unique values in Weather_Code Column 
bikes.weather_code.value_counts() 

1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: weather_code, dtype: int64

In [11]:
#Counting the Unique values in Season Column 
bikes.season.value_counts() 

0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: season, dtype: int64

In [12]:
# Creating a dict to map new column names (Changing column names)
new_cols_dict ={
    'timestamp':'time',
    'cnt':'count', 
    't1':'temp_real_C',
    't2':'temp_feels_like_C',
    'hum':'humidity_percent',
    'wind_speed':'wind_speed_kph',
    'weather_code':'weather',
    'is_holiday':'is_holiday',
    'is_weekend':'is_weekend',
    'season':'season'
}

# Renaming columns to specified column name 
bikes.rename(new_cols_dict, axis=1, inplace=True)


In [13]:
# Changing humidity values to percentage (previous values are not in correct format. Want values between 0 and 1)
bikes.humidity_percent = bikes.humidity_percent / 100 

In [14]:
# creating a season dicionary so I can map integer 0-3 to actual written values (Spring, Summer, Autumn, Winter) 
season_dict = {
    '0.0':'spring',
    '1.0':'summer',
    '2.0':'autumn',
    '3.0':'winter'
}

# creating a weather dictionary so we can map integers to actual written values (Clear, Scattered clouds, Broken clouds, cloudy, rain, rain with thunderstorm, snowfall) 
weather_dict = {
    '1.0':'Clear',
    '2.0':'Scattered clouds',
    '3.0':'Broken clouds',
    '4.0':'Cloudy',
    '7.0':'Rain',
    '10.0':'Rain with thunderstorm',
    '26.0':'Snowfall'
}

#utilizing map function to map the values to the new dicts created above 

# changing the seasons column data type to string
bikes.season = bikes.season.astype('str')
# mapping the values 0-3 to the actual written seasons
bikes.season = bikes.season.map(season_dict)

# changing the weather column data type to string
bikes.weather = bikes.weather.astype('str')
# mapping the values to the actual written weathers
bikes.weather = bikes.weather.map(weather_dict)

In [15]:
bikes.head()

,time,count,temp_real_C,temp_feels_like_C,humidity_percent,wind_speed_kph,weather,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,Broken clouds,0.0,1.0,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,Clear,0.0,1.0,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,Clear,0.0,1.0,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,Clear,0.0,1.0,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,Clear,0.0,1.0,winter


In [19]:
# Moving the Pandas DF to an Excel File for Data visuilzations 
bikes.to_excel('london_bikes_final.xlsx', sheet_name='Data')